# Метрики кластеризации

> Рассмотренные ниже метрики кластеризации применимы только в том случае,\
если есть размеченные данные, к каким кластерам принадлежат элементы.

In [1]:
from sklearn.metrics.cluster import (
    homogeneity_score, 
    completeness_score, 
    rand_score
)
from sklearn.metrics import v_measure_score

from warnings import simplefilter
simplefilter('ignore')

## Однородность кластеров (homogeneity score)

Показывает, насколько однородны получившиеся кластеры. \
Если в кластере оказались элементы из другого кластера, значение метрики уменьшается.

> Кластер считается однородным, если в нём содержатся объекты, \
принадлежащие только к одному кластеру.

Пусть: \
$n$ — общее количество объектов в выборке, \
$n_c$​​ — количество объектов класса $c$ (истинные классы, всего классов $C$ штук), ​\
$n_k$​ — количество объектов в кластере номер $k$ (предсказанные кластеры, всего кластеров $K$ штук), \
а ​$n_{ck}$​ — количество объектов из класса $c$ в кластере $k$.

Введём следующие величины:

$$ H(y_{true}) = - \sum_{c=1}^{C} P(y_{c}) log P(y_{c}), $$

где $P(y_{c}) = \frac{n_c}{n}$ — вероятность класса с или доля объектов в выборке, \
принадлежащих классу под номером $c$;

$$ H(y_{pred}) = - \sum_{k=1}^{K} P(y_{k}) log P(y_{k}), $$

где $P(y_{k}) = \frac{n_k}{n}$ — вероятность кластера $k$ или доля объектов в выборке, \
попавших в кластер под номером $k$;

$$ H(y_{true} | y_{pred}) = - \sum_{c=1}^{C} \sum_{k=1}^{K} P(y_{ck}) log (P(y_{ck})), $$

где $P(y_{ck}) = \frac{n_{ck}}{n}$ — вероятность класса $c$ в кластере $k$ \
или доля объектов в выборке, принадлежащих классу под номером $c$ и попавших в кластер под номером $k$.

Функция $H(y)$ называется **критерием информативности**, или **энтропией**.

То есть $H(y_{true})$ — энтропия истинных номеров кластеров (классов) для элементов, \
$H(y_{pred})$ — энтропия предсказанных номеров кластеров для элементов, \
а $H(y_{true}| y_{pred})$ — условная энтропия $y_{true}$ относительно $y_{pred}$.

Тогда **homogeneity** (однородность/гомогенность) будет рассчитываться по следующей формуле:

$$ homogeneity = 1 - \frac{H(y_{true} | y_{pred})}{H(y_{true})} $$

**Диапазон значений**\
1 — идеально однородные кластеры; \
0 —&nbsp;кластеры максимально разнородные.

### Реализация в sklearn

Функция в `sklearn.metrics.cluster`: `homogeneity_score`

In [2]:
# истинные метки классов
labels_true = [0, 0, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 1, 1]

# метрика однородности
homogeneity = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)

print(homogeneity)

1.0


Поменяем местами классы в редсказанных значениях.

In [3]:
# истинные метки классов
labels_true = [0, 0, 1, 1]
# предсказанные метки классов
labels_pred = [1, 1, 0, 0]

# метрика однородности
homogeneity = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)

print(homogeneity)

1.0


Метрика также равна 1, так как классы однородны.

Передадим данные, где один из объектов класса 1 предсказан как 0.

In [4]:
# истинные метки классов
labels_true = [0, 0, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 1]

# метрика однородности
homogeneity = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)

print(homogeneity)

0.31127812445913283


Значение метрики стало ниже.

Предположим, что мы в предсказании разделили данные на большее, чем 2 количество кластеров.

In [5]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 1, 1, 2, 2]

# метрика однородности
homogeneity = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)

print(homogeneity)

1.0000000000000002


Значение метрики 1, так как выделенный новый класс 2 однородный \
(в нем содержатся элементы изначально, принадлежащие к одному классу 1.)

Попробуем взять в класс 2 элементы из разных классов.

In [6]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 2, 1, 1, 1, 2]

# метрика однородности
homogeneity = homogeneity_score(labels_true=labels_true, labels_pred=labels_pred)

print(homogeneity)

0.7500000000000002


Значение метрики стало ниже.

## Полнота кластера (completeness score)

Показывает, насколько кластер заполнен объектами, которые в действительности должны принадлежать к этому кластеру.

Результат кластеризации удовлетворяет требованиям полноты, \
если все элементы данных, принадлежащие к одному классу, \
при кластеризации оказались в одном кластере.

Пусть:\
$n$ — общее количество объектов в выборке, \
$n_c$ — количество объектов класса (истинные классы, всего классов $C$ штук), \
$n_k$ — количество объектов в кластере номер $k$ (предсказанные кластеры, всего кластеров $K$ штук), \
а $n_{ck}$ — количество объектов из класса $c$ в кластере $k$.

Введём следующие величины:

$$ H(y_{true}) = - \sum_{c=1}^{C} P(y_{c}) log P(y_{c}), $$

где $P(y_{c}) = \frac{n_c}{n}$ — вероятность класса $c$ \
или доля объектов в выборке, принадлежащих классу под номером $c$;

$$ H(y_{pred}) = - \sum_{k=1}^{K} P(y_{k}) log P(y_{k}), $$

где $P(y_{k}) = \frac{n_k}{n}$ — вероятность кластера $k$ \
или доля объектов в выборке, попавших в кластер под номером $k$;

$$ H(y_{pred} | y_{true}) = - \sum_{k=1}^{K} \sum_{c=1}^{C} P(y_{ck}) log (P(y_{ck})), $$

где $P(y_{ck}) = \frac{n_{ck}}{n}$ — вероятность класса $c$ в кластере $k$ \
или доля объектов в выборке, принадлежащих классу под номером $c$ и попавших в кластер под номером $k$.

Функция $H(y)$ называется **критерием информативности**, или **энтропией**.

То есть \
$H(y_{true})$ — энтропия истинных номеров кластеров (классов) для элементов, \
$H(y_{pred})$ — энтропия предсказанных номеров кластеров для элементов, \
а $H(y_{pred}| y_{true})$ — условная энтропия $ y_{pred}$ относительно $y_{true}$.

Тогда **completeness** (полнота) будет рассчитываться по следующей формуле:

$$ completeness = 1 - \frac{H(y_{pred} | y_{true})}{H(y_{pred})} $$

**Диапазон значений**\
1 — идеальное значение; \
0 — объекты, которые должны образовывать один кластер, разделились на большее количество кластеров.

### Реализация в sklearn

Функция в `sklearn.metrics.cluster`: `completeness_score`

In [7]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 1, 1, 1, 1]

# метрика полноты
completeness = completeness_score(labels_true=labels_true, labels_pred=labels_pred)

print(completeness)

1.0


В предсказанных данных разделим класс 1 на два класса.

In [8]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 1, 1, 2, 2]

# метрика полноты
completeness = completeness_score(labels_true=labels_true, labels_pred=labels_pred)

print(completeness)

0.6666666666666669


Значение метрики уменьшилось.

## V-мера (V-Measure)

Комбинация метрик полноты и однородности кластеров.

V-мера рассчитывается по формуле:

$$ v = \frac{(1 + \beta) \times homogeneity \times completeness}{(\beta \times homogeneity + completeness)} $$

По умолчанию $\beta=1$, но это значение можно варьировать, если хочется дать разный вес разным свойствам.

- Если однородность кластеров важнее, чем их полнота, следует указать значение $\beta < 1$. \
Тогда значение $\beta \times homogeneity$ в знаменателе получится небольшим \
и тем самым будет сильнее влиять на значение $v$. \
Чем меньше $\beta \times homogeneity$, тем выше $v$.

- Если однородность кластеров не особо важна, но важно, 
- чтобы каждый кластер содержал максимальное количество похожих объектов, 
- тогда мы регулируем значение $\beta$ так, чтобы $\beta > 1$.

**Диапазон значений**\
1 — идеально полные и однородные кластеры; \
0 — полученные кластеры неоднородные, количество кластеров слишком большое.

- Данную метрику можно сравнить с метрикой **F-меры**, для задачи классификации. 

- Метрику однородности кластера при кластеризации можно сравнить с метрикой **precision** для задачи классификации:\
метрика однородности также показывает, насколько точно мы предсказали, к какому классу принадлежит объект. 

- Метрика полноты так же, как метрика **recall** из задачи классификации, показывает, \
насколько мы наполнили кластеры теми объектами, которые должны принадлежать к данным кластерам.

### Реализация в sklearn

Функция в `sklearn.metrics`: `v_measure_score`

In [9]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 1, 1, 1, 1]

# метрика полноты
v_measure = v_measure_score(labels_true=labels_true, labels_pred=labels_pred)

print(v_measure)

1.0


В предсказанных значениях разделим класс 1 на два.

In [10]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 1, 1, 2, 2]

# метрика полноты
v_measure = v_measure_score(labels_true=labels_true, labels_pred=labels_pred)

print(v_measure)

0.8000000000000003


Значение метрики меньше 1.

Теперь в редсказанных значениях переместим один из объектов класса 1 в класс 0.

In [11]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 0, 1, 1, 1]

# метрика полноты
v_measure = v_measure_score(labels_true=labels_true, labels_pred=labels_pred)

print(v_measure)

0.5615896365639194


Значение метрики также меньше 1.

## Индекс Рэнда

Показывает долю объектов датасета, которые мы правильно определили в кластер.

Данный индекс сравнивает предсказанный датасет и размеченные данные и подсчитывает, \
сколько образовалось пар объектов, которые оказались в одном кластере (*number of agreeing pairs*), \
среди предсказанных и размеченных данных.

$$ Rand \ Index = \frac{number \ of \ agreeing \ pairs}{number \ of \ pairs}, $$

где $number \ of \ pairs$ — общее количество пар, \
$number \ of \ agreeing \ pairs$ — количество пар, для которых предсказание и истинные значения совпали.\
По сути, данная метрика подсчитывает, для какой доли объектов датасета мы правильно определили кластер.

**Диапазон значений**\
1 — все объекты в предсказанном кластере попали в правильные кластеры.

### Реализация в sklearn

Функция в `sklearn.metrics.cluster`: `rand_score`

In [12]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 1, 1, 1, 1]

# считаем индекс Рэнда
rand_index = rand_score(labels_true, labels_pred)

print(rand_index)

1.0


Сделаем для одного объекта неверное предсказание.

In [13]:
# истинные метки классов
labels_true = [0, 0, 0, 0, 1, 1, 1, 1]
# предсказанные метки классов
labels_pred = [0, 0, 0, 0, 0, 1, 1, 1]

# считаем индекс Рэнда
rand_index = rand_score(labels_true, labels_pred)

print(rand_index)

0.75


Значение меньше 1.